<!-- TABS -->
# Create datatype

In [ ]:
# <testing:>
from superduperdb import superduper

db = superduper("mongomock://test")

In [ ]:
# <tab: Vector>
from superduperdb import vector

datatype = vector(shape=(3, ))
origin_data = [0.1, 0.2, 0.3]

In [ ]:
# <tab: Tensor>
from superduperdb.ext.torch import tensor
import torch

datatype = tensor(torch.float, shape=(32, 32, 3))
origin_data = torch.randn(32, 32, 3)

In [ ]:
# <tab: Array>
from superduperdb.ext.numpy import array
import numpy as np

datatype = array(dtype="float64", shape=(32, 32, 3))
origin_data = np.random.randn(32, 32, 3)

In [ ]:
# <tab: Image>
from superduperdb.ext.pillow import pil_image
import PIL.Image

!wget https://raw.githubusercontent.com/openai/CLIP/main/CLIP.png

datatype = pil_image
origin_data = PIL.Image.open("CLIP.png")

In [ ]:
# <tab: Audio>
!wget https://ccrma.stanford.edu/workshops/mir2014/audio/simpleLoop.wav

import librosa
from superduperdb.ext.numpy import array
import numpy as np
from IPython.display import Audio

datatype = array(dtype="float32", shape=(None, ))
origin_data, fs = librosa.load('simpleLoop.wav')

Audio(origin_data, rate=fs)

In [ ]:
# <tab: Video>
from superduperdb import DataType

# Create an instance of the Encoder with the identifier 'video_on_file' and load_hybrid set to False
datatype = DataType(
    identifier='video_on_file',
    encodable='artifact',
)

In [ ]:
# <tab: Custom-in-DB>

from superduperdb import DataType
import pandas as pd

def encoder(x, info=None):
    return x.to_json()

def decoder(x, info):
    return pd.read_json(x)
    
datatype = DataType(
    identifier="pandas",
    encoder=encoder,
    decoder=decoder
)
origin_data = pd.DataFrame([{"id": "1", "x": "xxx", "y": "yyy"}])

In [ ]:
# <tab: Custom-Artifact>

from superduperdb import DataType
import numpy as np
import pickle
def pickle_encode(object, info=None):
    return pickle.dumps(object)


def pickle_decode(b, info=None):
    return pickle.loads(b)


datatype = DataType(
    identifier="VectorSearchMatrix",
    encoder=pickle_encode,
    decoder=pickle_decode,
    encodable='artifact',
)

origin_data = np.random.randn(10000, 1024)

In [ ]:
# <testing>
db.add(datatype)

from superduperdb.backends.mongodb import Collection
from superduperdb import Document
collection = Collection("data")

print(origin_data)

db.execute(collection.insert_one(Document({"x": datatype(origin_data)})))

data = db.execute(collection.find_one())
print(data.unpack()["x"])